In [1]:
#!pip install requests
import pandas as pd
import requests

# Corrigindo tabelas clientes

In [2]:
df_clientes_l = pd.read_parquet('../landing/data_landing/idv_store_clientes_bd.parquet')
df_clientes_cf = pd.read_parquet('../landing/data_landing/idv_store_clientes_cf.parquet')

In [3]:
df_clientes_l.head(1)

,id_cliente,nome_completo,cpf_cnpj,email,telefone,data_nascimento,data_cadastro
0,1,Cliente Nome 1,1,cliente1@emailaleatorio.com,11900000001,NaN,2025-07-24 17:49:55


In [4]:
df_clientes_cf.head(1)

,Nº,Nome Completo,CPF,Data de Nascimento,Email,CEP,Número
0,1,Helena Ribeiro da Silva,847.192.508-37,1988-07-15,helena.silva@gmail.com,04538-132,455


In [5]:
df_merge_clientes = pd.merge(df_clientes_l, df_clientes_cf, left_on='id_cliente', right_on='Nº', how='left')
df_merge_clientes.head(1)

,id_cliente,nome_completo,cpf_cnpj,email,telefone,data_nascimento,data_cadastro,Nº,Nome Completo,CPF,Data de Nascimento,Email,CEP,Número
0,1,Cliente Nome 1,1,cliente1@emailaleatorio.com,11900000001,NaN,2025-07-24 17:49:55,1,Helena Ribeiro da Silva,847.192.508-37,1988-07-15,helena.silva@gmail.com,04538-132,455


In [6]:
df_clientes_merge = df_merge_clientes.drop(['nome_completo','cpf_cnpj','email','data_nascimento','Nº','CEP','Número'], axis=1)
df_clientes_merge.head(1)

,id_cliente,telefone,data_cadastro,Nome Completo,CPF,Data de Nascimento,Email
0,1,11900000001,2025-07-24 17:49:55,Helena Ribeiro da Silva,847.192.508-37,1988-07-15,helena.silva@gmail.com


In [7]:
df_clientes_b = df_clientes_merge.rename(columns={'CPF':'cpf_cnpj', 'Data de Nascimento':'data_nascimento','Email':'email','Nome Completo':'nome_completo'})
df_clientes_b.head(1)

,id_cliente,telefone,data_cadastro,nome_completo,cpf_cnpj,data_nascimento,email
0,1,11900000001,2025-07-24 17:49:55,Helena Ribeiro da Silva,847.192.508-37,1988-07-15,helena.silva@gmail.com


In [8]:
ordem_colunas_clientes = ['id_cliente','nome_completo','data_nascimento','cpf_cnpj','telefone','email','data_cadastro']
df_clientes_b = df_clientes_b.reindex(columns=ordem_colunas_clientes)
df_clientes_b.head(1)

,id_cliente,nome_completo,data_nascimento,cpf_cnpj,telefone,email,data_cadastro
0,1,Helena Ribeiro da Silva,1988-07-15,847.192.508-37,11900000001,helena.silva@gmail.com,2025-07-24 17:49:55


# Corrigindo tabelas endereços

In [9]:
df_enderecos_l = pd.read_parquet('../landing/data_landing/idv_store_enderecos_bd.parquet')

In [10]:
df_enderecos_l.head(1)

,id_endereco,id_cliente,logradouro,numero,complemento,bairro,cidade,estado,cep,tipo
0,1,1,Rua das Flores1,1,NaN,Centro,São Paulo,SP,01000-000,Principal


In [11]:
def buscar_cep(cep):
    url = f'https://viacep.com.br/ws/{cep}/json/'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'cep': cep, 'erro': True}

In [12]:
df_lista_ceps = df_merge_clientes['CEP'].dropna().unique().tolist()


In [13]:
dados_ceps = []
for cep in df_lista_ceps:
    dados_ceps.append(buscar_cep(cep))
    
df_dados_ceps = pd.DataFrame(dados_ceps)

In [14]:
df_dados_ceps.head(1)

,cep,logradouro,complemento,unidade,bairro,localidade,uf,estado,regiao,ibge,gia,ddd,siafi,erro
0,04538-132,Avenida Brigadeiro Faria Lima,de 3252 ao fim - lado par,,Itaim Bibi,São Paulo,SP,São Paulo,Sudeste,3550308,1004,11,7107,NaN


In [15]:
df_merge_clientes.head(1)

,id_cliente,nome_completo,cpf_cnpj,email,telefone,data_nascimento,data_cadastro,Nº,Nome Completo,CPF,Data de Nascimento,Email,CEP,Número
0,1,Cliente Nome 1,1,cliente1@emailaleatorio.com,11900000001,NaN,2025-07-24 17:49:55,1,Helena Ribeiro da Silva,847.192.508-37,1988-07-15,helena.silva@gmail.com,04538-132,455


In [16]:
df_clientes_f = df_merge_clientes[['id_cliente', 'CEP', 'Número']]
df_dados_ceps_f = df_dados_ceps[['cep','logradouro', 'complemento', 'unidade', 'bairro', 'localidade', 'uf', 'estado', 'regiao']]

df_merge_clientes_enderecos = pd.merge(df_clientes_f, df_dados_ceps_f, left_on='CEP', right_on='cep', how='left')

df_merge_clientes_enderecos.head(1)

,id_cliente,CEP,Número,cep,logradouro,complemento,unidade,bairro,localidade,uf,estado,regiao
0,1,04538-132,455,04538-132,Avenida Brigadeiro Faria Lima,de 3252 ao fim - lado par,,Itaim Bibi,São Paulo,SP,São Paulo,Sudeste


In [17]:
df_enderecos_f = df_enderecos_l[['id_endereco', 'id_cliente', 'tipo']]

df_enderecos_f2 = pd.merge(df_merge_clientes_enderecos, df_enderecos_f,  on='id_cliente')

df_enderecos_f2.head(1)

,id_cliente,CEP,Número,cep,logradouro,complemento,unidade,bairro,localidade,uf,estado,regiao,id_endereco,tipo
0,1,04538-132,455,04538-132,Avenida Brigadeiro Faria Lima,de 3252 ao fim - lado par,,Itaim Bibi,São Paulo,SP,São Paulo,Sudeste,1,Principal


In [18]:
df_enderecos_f3 = df_enderecos_f2.rename(columns={'Número':'numero','localidade':'cidade'})

In [19]:
ordem_colunas_enderecos = ['id_endereco', 'id_cliente','tipo', 'logradouro', 'numero', 'complemento',  'bairro', 'cidade', 'estado', 'uf', 'regiao']
df_enderecos_b = df_enderecos_f3.reindex(columns=ordem_colunas_enderecos)
df_enderecos_b

,id_endereco,id_cliente,tipo,logradouro,numero,complemento,bairro,cidade,estado,uf,regiao
0,1,1,Principal,Avenida Brigadeiro Faria Lima,455,de 3252 ao fim - lado par,Itaim Bibi,São Paulo,São Paulo,SP,Sudeste
1,2,2,Principal,Rua Lauro Muller,1204,,Botafogo,Rio de Janeiro,Rio de Janeiro,RJ,Sudeste
2,3,3,Principal,Rua 85,33,,Setor Sul,Goiânia,Goiás,GO,Centro-Oeste
3,4,4,Principal,Avenida Boa Viagem,2109,de 2176/2177 a 3077/3078,Boa Viagem,Recife,Pernambuco,PE,Nordeste
4,5,5,Principal,Rua Duque de Caxias,780,de 1190 ao fim - lado par,Centro Histórico,Porto Alegre,Rio Grande do Sul,RS,Sul
...,...,...,...,...,...,...,...,...,...,...,...
145,146,146,Principal,Avenida Washington Soares,111,até 3737 - lado ímpar,Edson Queiroz,Fortaleza,Ceará,CE,Nordeste
146,147,147,Principal,Praça Dom Jerônimo,113,,Centro,Sobral,Ceará,CE,Nordeste
147,148,148,Principal,Avenida Pedro Miranda,115,de 1836/1837 a 2050/2051,Pedreira,Belém,Pará,PA,Norte
148,149,149,Principal,Rua T 52,117,,Setor Bueno,Goiânia,Goiás,GO,Centro-Oeste


# Corrigindo tabelas produtos

In [20]:
df_produtos_l = pd.read_parquet('../landing/data_landing/idv_store_produtos_bd.parquet')
df_produtos_cf = pd.read_parquet('../landing/data_landing/idv_store_produtos_cf.parquet')

In [21]:
df_produtos_l.head(1)

,id_produto,id_categoria,nome,descricao,sku,preco_custo,preco_venda,unidade_medida,data_cadastro
0,1,4,Produto 1,NaN,SKU00000001,38.75,59.67,UN,2025-07-24 17:49:56


In [22]:
df_produtos_cf.head(1)

,id_produto,Categoria,Nome do Produto,Descrição do Produto
0,1,Materiais Básicos,Cimento CPII-F 50kg,"Cimento Portland de alta resistência, ideal pa..."


In [23]:
df_produtos_l_f = df_produtos_l[['id_produto', 'id_categoria', 'sku', 'preco_custo', 'preco_venda', 'unidade_medida', 'data_cadastro']]
df_produtos_cf_f = df_produtos_cf[['id_produto','Nome do Produto', 'Descrição do Produto']]

In [24]:
df_produtos_merge = pd.merge(df_produtos_l_f, df_produtos_cf_f, on='id_produto', how='left')
df_produtos_merge.head(1)

,id_produto,id_categoria,sku,preco_custo,preco_venda,unidade_medida,data_cadastro,Nome do Produto,Descrição do Produto
0,1,4,SKU00000001,38.75,59.67,UN,2025-07-24 17:49:56,Cimento CPII-F 50kg,"Cimento Portland de alta resistência, ideal pa..."


In [25]:
df_produtos_b = df_produtos_merge.rename(columns={'Nome do Produto':'nome','Descrição do Produto':'descricao'})

In [26]:
ordem_colunas_produtos = ['id_produto', 'id_categoria', 'nome', 'descricao', 'sku', 'preco_custo', 'preco_venda', 'unidade_medida', 'data_cadastro' ]

df_produtos_b = df_produtos_b.reindex(columns=ordem_colunas_produtos)

df_produtos_b.head(1)

,id_produto,id_categoria,nome,descricao,sku,preco_custo,preco_venda,unidade_medida,data_cadastro
0,1,4,Cimento CPII-F 50kg,"Cimento Portland de alta resistência, ideal pa...",SKU00000001,38.75,59.67,UN,2025-07-24 17:49:56


# Puxando as outras tabelas da camada landing

In [27]:
df_categorias_b = pd.read_parquet('../landing/data_landing/idv_store_categorias_bd.parquet')
df_estoque_b = pd.read_parquet('../landing/data_landing/idv_store_estoque_bd.parquet')
df_fornecedores_b = pd.read_parquet('../landing/data_landing/idv_store_fornecedores_bd.parquet')
df_funcionarios_b = pd.read_parquet('../landing/data_landing/idv_store_funcionarios_bd.parquet')
df_itens_pedidos_b = pd.read_parquet('../landing/data_landing/idv_store_itens_pedidos_bd.parquet')
df_pagamentos_b = pd.read_parquet('../landing/data_landing/idv_store_pagamentos_bd.parquet')
df_pedidos_b = pd.read_parquet('../landing/data_landing/idv_store_pedidos_bd.parquet')

# Salvando dataframes em parquet

In [28]:
df_categorias_b.to_parquet('data_bronze/idv_store_categorias_bd.parquet', index=False)
df_clientes_b.to_parquet('data_bronze/idv_store_clientes_bd.parquet', index=False)
df_enderecos_b.to_parquet('data_bronze/idv_store_enderecos_bd.parquet', index=False)
df_estoque_b.to_parquet('data_bronze/idv_store_estoque_bd.parquet', index=False)
df_fornecedores_b.to_parquet('data_bronze/idv_store_fornecedores_bd.parquet', index=False)
df_funcionarios_b.to_parquet('data_bronze/idv_store_funcionarios_bd.parquet', index=False)
df_itens_pedidos_b.to_parquet('data_bronze/idv_store_itens_pedidos_bd.parquet', index=False)
df_pagamentos_b.to_parquet('data_bronze/idv_store_pagamentos_bd.parquet', index=False)
df_pedidos_b.to_parquet('data_bronze/idv_store_pedidos_bd.parquet', index=False)
df_produtos_b.to_parquet('data_bronze/idv_store_produtos_bd.parquet', index=False)